# MA_DL : Codierung 64bit

In [1]:
!pip install facenet-pytorch # due to issues with the google cloud service

In [2]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

dtype = torch.cuda.FloatTensor

## Model

In [3]:
class MultiLabel(nn.Sequential):
    def __init__(self, input_dim=512, output_dim=64):
        super(MultiLabel, self).__init__()
        self.l1 = nn.Linear(input_dim, 512)
        self.l2 = nn.Linear(512, 300)
        self.l5 = nn.Linear(300, output_dim)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return torch.sigmoid(x)

In [4]:
model = MultiLabel()
model.cuda()

MultiLabel(
  (l1): Linear(in_features=512, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=300, bias=True)
  (l5): Linear(in_features=300, out_features=64, bias=True)
)

## Data

In [5]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()
transform = transforms.Compose([
    scaler, 
    to_tensor,
#    normalize
])

In [6]:
train_ds = datasets.ImageFolder('../../data/small_data/train/', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds, batch_size=1, num_workers=4)

valid_ds = datasets.ImageFolder('../../data/small_data/valid/', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds, batch_size=1, num_workers=4)

## Gesichtserkennung

In [7]:
mtcnn = MTCNN(
#    image_size=160, margin=0, min_face_size=20,
#        thresholds=[0.6, 0.7, 0.7],
#        factor=0.709,
#        prewhiten=True,
    keep_all=True,
    device=torch.device('cuda')
    )

## Embeddings 

In [8]:
def get_vectors(imgs):
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(imgs).type(dtype)  
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(imgs.shape[0], 512)    

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.squeeze())    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    resnet(t_img)    
    # 7. Detach our copy function from the layer
    h.remove()    
    # 8. Return the feature vector
    return my_embedding

In [9]:
# Load the pretrained model
resnet = models.resnet18(pretrained=True)
resnet.cuda()
# Use the model object to select the desired layer
layer = resnet._modules.get('avgpool')
_ = resnet.eval()

## Training

In [10]:
def calculate_mistakes(pred, target):
    #pred = pred[0]
#    print('calculating mistakes : ')
#    print('   pred: ' + str(pred.shape))
#    print('   target: ' + str(target.shape))
    
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [11]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5) #weight_decay=1e-4 

In [12]:
import matplotlib.pyplot as plt

In [15]:
epochs = 400
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    # activate train mode
    model.train()

    for index, (data, target) in enumerate(train_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
 
 #       if faces is None:
 #           print('no face')
 #           continue
 #       else:
 #           print('face found')
                
        target_t = torch.cuda.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])

        emb = get_vectors(faces)    
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        # forward
#        print('datav: ' + str(data_v.shape))
        pred = model.forward(data_v)
#        print('pred: ' + str(pred.shape))
        # zero grads
        optimizer.zero_grad()
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    # activate eval mode
    model.eval()
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        emb = get_vectors(faces) 
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))

Epoch : 0
Train loss : 24.235774993896484
Average errors : 0.4098360655737705
Valid loss : 7.468125343322754
Average valid errors : 4.214285714285714
Epoch : 1
Train loss : 24.02813148498535
Average errors : 0.39344262295081966
Valid loss : 7.412400245666504
Average valid errors : 4.142857142857143
Epoch : 2
Train loss : 23.836816787719727
Average errors : 0.3879781420765027
Valid loss : 7.384714603424072
Average valid errors : 4.166666666666667
Epoch : 3
Train loss : 23.642560958862305
Average errors : 0.3797814207650273
Valid loss : 7.335810661315918
Average valid errors : 4.095238095238095
Epoch : 4
Train loss : 23.456445693969727
Average errors : 0.37158469945355194
Valid loss : 7.263070106506348
Average valid errors : 3.9761904761904763
Epoch : 5
Train loss : 23.243431091308594
Average errors : 0.3469945355191257
Valid loss : 7.247687816619873
Average valid errors : 4.0
Epoch : 6
Train loss : 23.061914443969727
Average errors : 0.3524590163934426
Valid loss : 7.184078216552734
Ave

Train loss : 16.647653579711914
Average errors : 0.12295081967213115
Valid loss : 5.448885440826416
Average valid errors : 2.7857142857142856
Epoch : 48
Train loss : 16.504711151123047
Average errors : 0.12295081967213115
Valid loss : 5.416210174560547
Average valid errors : 2.761904761904762
Epoch : 49
Train loss : 16.386207580566406
Average errors : 0.11748633879781421
Valid loss : 5.380786895751953
Average valid errors : 2.738095238095238
Epoch : 50
Train loss : 16.26342010498047
Average errors : 0.11202185792349727
Valid loss : 5.324463367462158
Average valid errors : 2.6904761904761907
Epoch : 51
Train loss : 16.131671905517578
Average errors : 0.1092896174863388
Valid loss : 5.296465873718262
Average valid errors : 2.6666666666666665
Epoch : 52
Train loss : 16.014963150024414
Average errors : 0.10655737704918032
Valid loss : 5.263277530670166
Average valid errors : 2.6904761904761907
Epoch : 53
Train loss : 15.894411087036133
Average errors : 0.10109289617486339
Valid loss : 5.21

Valid loss : 3.9372246265411377
Average valid errors : 1.9761904761904763
Epoch : 95
Train loss : 11.701078414916992
Average errors : 0.01912568306010929
Valid loss : 3.91097092628479
Average valid errors : 1.9047619047619047
Epoch : 96
Train loss : 11.62035083770752
Average errors : 0.01912568306010929
Valid loss : 3.887392044067383
Average valid errors : 1.8571428571428572
Epoch : 97
Train loss : 11.534154891967773
Average errors : 0.01912568306010929
Valid loss : 3.8574037551879883
Average valid errors : 1.880952380952381
Epoch : 98
Train loss : 11.456110000610352
Average errors : 0.01912568306010929
Valid loss : 3.8241310119628906
Average valid errors : 1.8333333333333333
Epoch : 99
Train loss : 11.37653636932373
Average errors : 0.01912568306010929
Valid loss : 3.798631191253662
Average valid errors : 1.8333333333333333
Epoch : 100
Train loss : 11.2979736328125
Average errors : 0.01912568306010929
Valid loss : 3.7738723754882812
Average valid errors : 1.8571428571428572
Epoch : 10

Valid loss : 2.844097137451172
Average valid errors : 1.1666666666666667
Epoch : 142
Train loss : 8.58610725402832
Average errors : 0.01092896174863388
Valid loss : 2.8340628147125244
Average valid errors : 1.1904761904761905
Epoch : 143
Train loss : 8.529726028442383
Average errors : 0.01092896174863388
Valid loss : 2.806884288787842
Average valid errors : 1.1904761904761905
Epoch : 144
Train loss : 8.486552238464355
Average errors : 0.01092896174863388
Valid loss : 2.7866110801696777
Average valid errors : 1.1666666666666667
Epoch : 145
Train loss : 8.428725242614746
Average errors : 0.01092896174863388
Valid loss : 2.7677929401397705
Average valid errors : 1.1428571428571428
Epoch : 146
Train loss : 8.381003379821777
Average errors : 0.00819672131147541
Valid loss : 2.7533440589904785
Average valid errors : 1.1666666666666667
Epoch : 147
Train loss : 8.324832916259766
Average errors : 0.01092896174863388
Valid loss : 2.7308123111724854
Average valid errors : 1.1428571428571428
Epoch

Train loss : 6.563097953796387
Average errors : 0.00273224043715847
Valid loss : 2.0703859329223633
Average valid errors : 0.6666666666666666
Epoch : 189
Train loss : 6.527990341186523
Average errors : 0.00273224043715847
Valid loss : 2.061682939529419
Average valid errors : 0.6666666666666666
Epoch : 190
Train loss : 6.490503311157227
Average errors : 0.00273224043715847
Valid loss : 2.0413243770599365
Average valid errors : 0.6666666666666666
Epoch : 191
Train loss : 6.454697608947754
Average errors : 0.00273224043715847
Valid loss : 2.025972604751587
Average valid errors : 0.6666666666666666
Epoch : 192
Train loss : 6.423096179962158
Average errors : 0.00273224043715847
Valid loss : 2.0133979320526123
Average valid errors : 0.6666666666666666
Epoch : 193
Train loss : 6.388456344604492
Average errors : 0.00273224043715847
Valid loss : 1.9978057146072388
Average valid errors : 0.6666666666666666
Epoch : 194
Train loss : 6.3494391441345215
Average errors : 0.00273224043715847
Valid los

Valid loss : 1.5009511709213257
Average valid errors : 0.5238095238095238
Epoch : 235
Train loss : 5.136652946472168
Average errors : 0.00273224043715847
Valid loss : 1.48914635181427
Average valid errors : 0.5238095238095238
Epoch : 236
Train loss : 5.11080265045166
Average errors : 0.00273224043715847
Valid loss : 1.4821650981903076
Average valid errors : 0.5238095238095238
Epoch : 237
Train loss : 5.0866241455078125
Average errors : 0.00273224043715847
Valid loss : 1.4717251062393188
Average valid errors : 0.5238095238095238
Epoch : 238
Train loss : 5.060670852661133
Average errors : 0.00273224043715847
Valid loss : 1.4571000337600708
Average valid errors : 0.5
Epoch : 239
Train loss : 5.034977912902832
Average errors : 0.00273224043715847
Valid loss : 1.449317216873169
Average valid errors : 0.5238095238095238
Epoch : 240
Train loss : 5.011940002441406
Average errors : 0.00273224043715847
Valid loss : 1.4391311407089233
Average valid errors : 0.5
Epoch : 241
Train loss : 4.98814678

Train loss : 4.065623760223389
Average errors : 0.0
Valid loss : 1.031999945640564
Average valid errors : 0.23809523809523808
Epoch : 286
Train loss : 4.04757833480835
Average errors : 0.0
Valid loss : 1.025514006614685
Average valid errors : 0.23809523809523808
Epoch : 287
Train loss : 4.031532287597656
Average errors : 0.0
Valid loss : 1.0170458555221558
Average valid errors : 0.23809523809523808
Epoch : 288
Train loss : 4.01209831237793
Average errors : 0.0
Valid loss : 1.0132098197937012
Average valid errors : 0.23809523809523808
Epoch : 289
Train loss : 3.9962503910064697
Average errors : 0.0
Valid loss : 1.003861665725708
Average valid errors : 0.23809523809523808
Epoch : 290
Train loss : 3.9775607585906982
Average errors : 0.0
Valid loss : 0.9934791922569275
Average valid errors : 0.23809523809523808
Epoch : 291
Train loss : 3.96048641204834
Average errors : 0.0
Valid loss : 0.9899508357048035
Average valid errors : 0.23809523809523808
Epoch : 292
Train loss : 3.94305157661438
A

Train loss : 3.2858688831329346
Average errors : 0.0
Valid loss : 0.7081081867218018
Average valid errors : 0.09523809523809523
Epoch : 337
Train loss : 3.2734758853912354
Average errors : 0.0
Valid loss : 0.7002422213554382
Average valid errors : 0.09523809523809523
Epoch : 338
Train loss : 3.260284423828125
Average errors : 0.0
Valid loss : 0.6965205073356628
Average valid errors : 0.09523809523809523
Epoch : 339
Train loss : 3.2473998069763184
Average errors : 0.0
Valid loss : 0.6916159391403198
Average valid errors : 0.09523809523809523
Epoch : 340
Train loss : 3.2359910011291504
Average errors : 0.0
Valid loss : 0.6850709915161133
Average valid errors : 0.09523809523809523
Epoch : 341
Train loss : 3.221531391143799
Average errors : 0.0
Valid loss : 0.6812195777893066
Average valid errors : 0.09523809523809523
Epoch : 342
Train loss : 3.2100448608398438
Average errors : 0.0
Valid loss : 0.6744092106819153
Average valid errors : 0.09523809523809523
Epoch : 343
Train loss : 3.1967654

Train loss : 2.6791069507598877
Average errors : 0.0
Valid loss : 0.46625208854675293
Average valid errors : 0.07142857142857142
Epoch : 392
Train loss : 2.6702136993408203
Average errors : 0.0
Valid loss : 0.46194714307785034
Average valid errors : 0.07142857142857142
Epoch : 393
Train loss : 2.660189151763916
Average errors : 0.0
Valid loss : 0.459178626537323
Average valid errors : 0.07142857142857142
Epoch : 394
Train loss : 2.651214122772217
Average errors : 0.0
Valid loss : 0.4551298916339874
Average valid errors : 0.07142857142857142
Epoch : 395
Train loss : 2.6422786712646484
Average errors : 0.0
Valid loss : 0.4509117007255554
Average valid errors : 0.07142857142857142
Epoch : 396
Train loss : 2.6326613426208496
Average errors : 0.0
Valid loss : 0.44890958070755005
Average valid errors : 0.07142857142857142
Epoch : 397
Train loss : 2.6236183643341064
Average errors : 0.0
Valid loss : 0.44495677947998047
Average valid errors : 0.07142857142857142
Epoch : 398
Train loss : 2.6150

In [16]:
torch.save(model.state_dict(), 'dl64b_v1_800.pt')